## Spam Text Messages Categorization using K-Means Clustering

#### Data Description

In [ ]:
import numpy as np 
import pandas as pd 
import csv
import os

for dirname, _, filenames in os.walk('/CSC172/K Means Project/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
pd.set_option('max_colwidth', 2000)
df = pd.read_csv("D:/Documents/4th year files/CSC172/K Means Project/SPAM_SMS.csv", index_col=0, quoting=csv.QUOTE_NONNUMERIC, encoding="utf-8")

In [3]:
df.shape

(948, 4)

In [4]:
df.columns

Index(['hashed_celphone_number', 'date', 'text', 'carrier'], dtype='object')

In [5]:
df.dtypes

hashed_celphone_number    object
date                      object
text                      object
carrier                   object
dtype: object

In [6]:
df.head(10)

,hashed_celphone_number,date,text,carrier
masked_celphone_number,,,,
+63998****809,6c8cee87-7025-3b11-b7f4-2445f62c2e8c,2018-01-23 04:46:26.719,"A new and ""Green"" township will be rising in the South, MAPLE GROVE.\n\nFor the first time, Megaworld is offering a chance of a lifetime to acquire the most prime COMMERCIAL LOT(s) in the rapidly developing business district of Gen. Trias Cavite.\n\nLot sizes range from 440 to 1,000 sqm (with option to combine)\n\nLimited inventory only. Get yours at PRE-SELLING prices TODAY! We are now accepting reservations.\n\nFor more details and information, please contact:\n\nAndrew @ ++63917****689",Smart
+63908****673,5a59adcb-ba3d-3b04-8dcc-3e39cc3f4974,2018-01-24 12:44:09.178,"New OFFICE space for sale in Ortigas Avenue near ""The Grove""\n\nFor inquiries, please text your email.\nText No to Stop.\n\nThank you,\nJenn Sam",Smart
+63917****194,a14c754f-80b2-3797-8dde-0c3232e2d1df,2018-02-01 20:55:48.554,"Celebrate the season of love with a sweet staycation and an intimate dinner for two by the poolside at your urban oasis, Edsa Shangri-La, Manila. Call 633 8888.",Globe PostPaid
+63920****952,132b9c54-4f79-33c3-b442-12d2318fb5d5,2018-02-28 10:37:26.541,Shangri-La Updates:\n\nRISEmakati\n\n-One-of-a-kind residential & commercial development\n-Premier outstanding amenities\n-Prices start at 5M\n\nInquiries are welcome!\n\nPls. reply w/ your name & email add for more details\n\nContact Us:\ninfo@shangrilapropertiesinc.com\n\nwww.shangrilapropertiesinc.com/rise-makati,Smart
+63907****270,28d44ab6-02a5-3eb6-8037-db43abbe8a47,2018-03-07 06:28:35.858,"Co in Makati needs Software Java Developer, knowledgeable in JAVA, in design, development & implementation of software applications , can start asap, kimphil28@gmail.com",TNT
+63946****050,86c4de11-cc48-35df-b84d-3e42720332f6,2018-03-22 10:45:57.147,"Co in Makati needs Software Java Developer, Bachelors degree, knowledgeable in JAVA, in design, development & implementation of software applications , can start asap, kimphil28@gmail.com",TNT
+63965****768,92570f5a-4726-3e64-9ea3-fc4b525e81da,2018-04-19 11:11:20.770,"Co in Makati needs Software Java Developer, Bachelors degree, knowledgeable in JAVA, in design, development & implementation of software applications , kimphil28@gmail.com",Globe/TM
+63917****194,a14c754f-80b2-3797-8dde-0c3232e2d1df,2018-04-25 20:41:34.001,"Dine, swim, and get away for a day without having to leave the city at Edsa Shangri-La, Manila's HEAT, The Sunday Side, for Php2,400net per person. Call 6338888",Globe PostPaid
+63998****809,6c8cee87-7025-3b11-b7f4-2445f62c2e8c,2018-04-29 02:25:24.016,"The LAST pre-selling low-rise condo in BGC is now available in Special Low DP, 0% Int terms.\n\nExperience low density living with private elevator access per unit that promises overwhelming views of Forbes Park and BGC skyline at THE ALBANY. VERY RARE FIND, LIMITED UNITS await you. Choose from spacious 2BR to 3BR units with luxe finishing.\n\n*Very Prime Pre-selling Commercial lots at Maple Grove in Gen Trias. Cavite for Sale!\n\nFor inquiries, please contact ANDREW @ +63917****689",Smart


In [7]:
df.describe()

,hashed_celphone_number,date,text,carrier
count,948,948,948,948
unique,933,934,877,10
top,6c8cee87-7025-3b11-b7f4-2445f62c2e8c,2023-09-02 17:58:14.125,<<Content not supported.>>,TNT
freq,4,2,13,346


In [8]:
from ydata_profiling import ProfileReport
ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Data Cleaning

In [10]:
import pandas as pd
import re

In [9]:
# Load your dataset 
data = pd.read_csv('SPAM_SMS.csv')

In [11]:
# Focus only on the 'text' column, drop the other columns
data = data[['text']]

# Data Cleaning
# Normalize Text: Convert all text to lowercase
data['text'] = data['text'].str.lower()

# Remove Punctuation
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Strip Extra Spaces
data['text'] = data['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Save the cleaned data to a new file
data.to_csv('cleaned_data.csv', index=False)

print("Data cleaning completed and saved as 'cleaned_data.csv'")


Data cleaning completed and saved as 'cleaned_data.csv'


#### Tokenization and Removal of Stopwords

In [21]:
import spacy
import pandas as pd

# Load the SpaCy model for English
nlp = spacy.load("en_core_web_sm")

# Load your dataset
data = pd.read_csv('cleaned_data.csv')  # Assuming you've already cleaned the 'text' column

# Load the stopword files (replace with your actual file paths)
with open('english-stopwords.txt', 'r') as file:
    english_stopwords = set(file.read().splitlines())

with open('genediazjr-tagalog.txt', 'r') as file:
    filipino_stopwords = set(file.read().splitlines())

# Combine English and Filipino stopwords into one set for easier filtering
stopwords_combined = english_stopwords.union(filipino_stopwords)

# Tokenization and Stopword Removal using SpaCy
def preprocess_text(text):
    # Tokenize the text with SpaCy
    doc = nlp(text)

    # Remove stopwords (both English and Filipino), and non-alphabetic tokens
    tokens = [token.text for token in doc if token.text.lower() not in stopwords_combined and token.is_alpha]

    return tokens

# Apply tokenization and stopword removal to the 'text' column
data['tokens'] = data['text'].apply(preprocess_text)

# Save the processed data with tokens to a new file
data.to_csv('processed_data.csv', index=False)

print("Tokenization and stopword removal completed and saved as 'processed_data.csv'")


Tokenization and stopword removal completed and saved as 'processed_data.csv'


In [22]:
pf = pd.read_csv("D:/Documents/4th year files/CSC172/K Means Project/processed_data.csv", index_col=0, quoting=csv.QUOTE_NONNUMERIC, encoding="utf-8")

In [23]:
from ydata_profiling import ProfileReport
ProfileReport(pf)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]